# **4 – Model Developmemt**

## Objectives
* Build and train a CNN to classify cherry leaf images as either healthy or diseased
* Evaluate model performance and save it for deployment

## Inputs
* Train/val/test folders from inputs/dataset

## Outputs
- Trained CNN model
- Accuracy/loss plots
- Evaluation results on test set


---

# Change working directory

Change the working directory from its current folder to its parent folder

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\amyno\\OneDrive\\Documents\\CherryLeafProject\\milestone-project-mildew-detection-in-cherry-leaves\\jupyter_notebooks'

Make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\amyno\\OneDrive\\Documents\\CherryLeafProject\\milestone-project-mildew-detection-in-cherry-leaves'

---